# 🏆 Competition Data EDA -

Exploratory Data Analysis chi tiết cho FreshRetail competition dataset.

## Mục tiêu
- Hiểu cấu trúc và đặc điểm của competition data
- Phân tích patterns và seasonality
- Identify data quality issues
- Prepare insights cho feature engineering

## Dataset Info
- **Name**: FreshRetailNet-50K
- **Type**: Hourly sales data
- **Competition**: Time-series forecasting challenge


In [1]:
# Setup
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Add project root to path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.config import setup_project_path, get_dataset_config, setup_logging
from src.pipelines._01_load_data import load_data
from src.utils.validation import comprehensive_validation

setup_project_path()
setup_logging()

# Import display for Jupyter notebooks
try:
    from IPython.display import display
except ImportError:
    # Fallback if not in Jupyter
    display = print

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)


Added D:\datastorm\E-Grocery_Forecaster\src to sys.path


## 1. Load Competition Data


In [2]:
# Load data
dataframes, config = load_data()

print("=" * 70)
print("FRESHRETAIL COMPETITION DATASET")
print("=" * 70)
print(f"Dataset: {config['name']}")
print(f"Temporal Unit: {config['temporal_unit']}")
print(f"Time Column: {config['time_column']}")
print(f"Target Column: {config['target_column']}")
print(f"\nDataFrames loaded: {len(dataframes)}")

# Get main sales dataframe
sales_df = dataframes.get('sales') or dataframes.get('freshretail_train')
if sales_df is None:
    # Try to find any dataframe with sales data
    for name, df in dataframes.items():
        if 'sales' in name.lower() or 'train' in name.lower():
            sales_df = df
            break

if sales_df is not None:
    print(f"\nMain Sales DataFrame: {sales_df.shape}")
    print(f"Columns: {list(sales_df.columns)}")
    display(sales_df.head())
else:
    print("⚠️ Sales dataframe not found!")


2025-11-13 01:31:42,490 - src.pipelines._01_load_data - INFO - ======================================================================
2025-11-13 01:31:42,492 - src.pipelines._01_load_data - INFO - [PIPELINE STEP 1: LOAD DATA]
2025-11-13 01:31:42,493 - src.pipelines._01_load_data - INFO - Active Dataset: FreshRetailNet-50K
2025-11-13 01:31:42,493 - src.pipelines._01_load_data - INFO - Data Directory: D:\datastorm\E-Grocery_Forecaster\data\2_raw
2025-11-13 01:31:42,494 - src.pipelines._01_load_data - INFO - ======================================================================
2025-11-13 01:31:42,495 - src.pipelines._01_load_data - INFO - Loading FreshRetailNet-50K dataset...
2025-11-13 01:31:42,495 - src.pipelines._01_load_data - WARNING -   File not found: sales_hourly.parquet/csv
2025-11-13 01:31:42,496 - src.pipelines._01_load_data - INFO -   Loading freshretail_train.parquet...
2025-11-13 01:31:45,969 - src.pipelines._01_load_data - INFO - ✓ Converted 'dt' to 'hour_timestamp'
2025-1

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## 2. Dataset Overview & Statistics


In [ ]:
if sales_df is not None:
    print("=" * 70)
    print("DATASET STATISTICS")
    print("=" * 70)
    
    # Basic info
    print(f"\nShape: {sales_df.shape}")
    print(f"Memory usage: {sales_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"Data types:\n{sales_df.dtypes}")
    
    # Missing values
    print(f"\nMissing Values:")
    missing = sales_df.isnull().sum()
    missing_pct = (missing / len(sales_df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    })
    display(missing_df[missing_df['Missing Count'] > 0])
    
    # Unique values
    print(f"\nUnique Values:")
    for col in sales_df.columns:
        n_unique = sales_df[col].nunique()
        print(f"  {col}: {n_unique} unique values")
    
    # Time range
    time_col = config['time_column']
    if time_col in sales_df.columns:
        print(f"\nTime Range:")
        print(f"  Min: {sales_df[time_col].min()}")
        print(f"  Max: {sales_df[time_col].max()}")
        print(f"  Span: {sales_df[time_col].max() - sales_df[time_col].min()}")


## 3. Target Variable Analysis


In [ ]:
if sales_df is not None:
    target_col = config['target_column']
    
    if target_col in sales_df.columns:
        print("=" * 70)
        print("TARGET VARIABLE ANALYSIS")
        print("=" * 70)
        
        target_data = sales_df[target_col]
        
        print(f"\nBasic Statistics:")
        print(target_data.describe())
        
        print(f"\nDistribution:")
        print(f"  Mean: {target_data.mean():.2f}")
        print(f"  Median: {target_data.median():.2f}")
        print(f"  Std: {target_data.std():.2f}")
        print(f"  Min: {target_data.min():.2f}")
        print(f"  Max: {target_data.max():.2f}")
        print(f"  Skewness: {target_data.skew():.2f}")
        print(f"  Kurtosis: {target_data.kurtosis():.2f}")
        
        # Zero values
        zero_count = (target_data == 0).sum()
        zero_pct = zero_count / len(target_data) * 100
        print(f"\nZero Values: {zero_count} ({zero_pct:.2f}%)")
        
        # Visualize distribution
        try:
            from IPython.display import display
        except ImportError:
            display = print
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Distribution', 'Log Distribution', 'Box Plot', 'Q-Q Plot'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )
        
        # Histogram
        fig.add_trace(
            go.Histogram(x=target_data, nbinsx=50, name='Distribution'),
            row=1, col=1
        )
        
        # Log histogram
        log_target = np.log1p(target_data[target_data > 0])
        fig.add_trace(
            go.Histogram(x=log_target, nbinsx=50, name='Log Distribution'),
            row=1, col=2
        )
        
        # Box plot
        fig.add_trace(
            go.Box(y=target_data, name='Box Plot'),
            row=2, col=1
        )
        
        fig.update_layout(height=800, showlegend=False, title_text="Target Variable Analysis")
        fig.show()
    else:
        print(f"⚠️ Target column '{target_col}' not found in dataframe")


## 4. Time Series Patterns


In [ ]:
if sales_df is not None and config['temporal_unit'] == 'hour':
    time_col = config['time_column']
    target_col = config['target_column']
    
    # Convert to datetime if needed
    if sales_df[time_col].dtype != 'datetime64[ns]':
        try:
            sales_df[time_col] = pd.to_datetime(sales_df[time_col])
        except:
            pass
    
    # Aggregate by day for visualization
    sales_df['date'] = sales_df[time_col].dt.date
    daily_sales = sales_df.groupby('date')[target_col].sum().reset_index()
    daily_sales['date'] = pd.to_datetime(daily_sales['date'])
    
    # Plot time series
    fig = px.line(daily_sales, x='date', y=target_col,
                  title='Daily Sales Over Time',
                  labels={'date': 'Date', target_col: 'Total Sales'})
    fig.update_layout(height=500)
    fig.show()
    
    # Extract time features
    sales_df['hour'] = sales_df[time_col].dt.hour
    sales_df['day_of_week'] = sales_df[time_col].dt.dayofweek
    sales_df['day_of_month'] = sales_df[time_col].dt.day
    
    # Hourly pattern
    hourly_pattern = sales_df.groupby('hour')[target_col].mean()
    fig2 = px.bar(x=hourly_pattern.index, y=hourly_pattern.values,
                  title='Average Sales by Hour of Day',
                  labels={'x': 'Hour', 'y': 'Average Sales'})
    fig2.update_layout(height=400)
    fig2.show()
    
    # Day of week pattern
    dow_pattern = sales_df.groupby('day_of_week')[target_col].mean()
    dow_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    fig3 = px.bar(x=[dow_names[i] for i in dow_pattern.index], 
                  y=dow_pattern.values,
                  title='Average Sales by Day of Week',
                  labels={'x': 'Day of Week', 'y': 'Average Sales'})
    fig3.update_layout(height=400)
    fig3.show()
    
elif sales_df is not None and config['temporal_unit'] == 'week':
    time_col = config['time_column']
    target_col = config['target_column']
    
    # Weekly aggregation
    weekly_sales = sales_df.groupby(time_col)[target_col].sum().reset_index()
    weekly_sales = weekly_sales.sort_values(time_col)
    
    # Plot
    fig = px.line(weekly_sales, x=time_col, y=target_col,
                  title='Weekly Sales Over Time',
                  labels={time_col: 'Week', target_col: 'Total Sales'})
    fig.update_layout(height=500)
    fig.show()


## 5. Product & Store Analysis


In [ ]:
if sales_df is not None:
    target_col = config['target_column']
    
    # Product analysis
    if 'PRODUCT_ID' in sales_df.columns or 'product_id' in sales_df.columns:
        product_col = 'PRODUCT_ID' if 'PRODUCT_ID' in sales_df.columns else 'product_id'
        
        product_sales = sales_df.groupby(product_col)[target_col].sum().sort_values(ascending=False)
        
        print("=" * 70)
        print("PRODUCT ANALYSIS")
        print("=" * 70)
        print(f"Total Products: {sales_df[product_col].nunique()}")
        print(f"\nTop 10 Products by Sales:")
        display(product_sales.head(10))
        
        # Visualize
        fig = px.bar(x=product_sales.head(20).index.astype(str), 
                     y=product_sales.head(20).values,
                     title='Top 20 Products by Total Sales',
                     labels={'x': 'Product ID', 'y': 'Total Sales'})
        fig.update_layout(height=500, xaxis_tickangle=-45)
        fig.show()
    
    # Store analysis
    if 'STORE_ID' in sales_df.columns or 'store_id' in sales_df.columns:
        store_col = 'STORE_ID' if 'STORE_ID' in sales_df.columns else 'store_id'
        
        store_sales = sales_df.groupby(store_col)[target_col].sum().sort_values(ascending=False)
        
        print("\n" + "=" * 70)
        print("STORE ANALYSIS")
        print("=" * 70)
        print(f"Total Stores: {sales_df[store_col].nunique()}")
        print(f"\nTop 10 Stores by Sales:")
        display(store_sales.head(10))
        
        # Visualize
        fig = px.bar(x=store_sales.head(20).index.astype(str), 
                     y=store_sales.head(20).values,
                     title='Top 20 Stores by Total Sales',
                     labels={'x': 'Store ID', 'y': 'Total Sales'})
        fig.update_layout(height=500)
        fig.show()
    
    # Product-Store combinations
    if ('PRODUCT_ID' in sales_df.columns or 'product_id' in sales_df.columns) and \
       ('STORE_ID' in sales_df.columns or 'store_id' in sales_df.columns):
        combo_cols = [product_col, store_col]
        combo_sales = sales_df.groupby(combo_cols)[target_col].sum().sort_values(ascending=False)
        
        print(f"\nTotal Product-Store Combinations: {len(combo_sales)}")
        print(f"Average Sales per Combination: {combo_sales.mean():.2f}")
        print(f"Median Sales per Combination: {combo_sales.median():.2f}")


## 6. Data Quality Assessment


In [ ]:
if sales_df is not None:
    print("=" * 70)
    print("DATA QUALITY ASSESSMENT")
    print("=" * 70)
    
    # Run comprehensive validation
    validation_results = comprehensive_validation(sales_df, verbose=True)
    
    quality_score = validation_results.get('quality_score', 0)
    print(f"\nOverall Quality Score: {quality_score}/100")
    
    if quality_score >= 90:
        print("✅ EXCELLENT - Data quality is very good")
    elif quality_score >= 75:
        print("⚠️ GOOD - Data quality is acceptable")
    else:
        print("❌ POOR - Data quality needs attention")
    
    # Check for specific issues
    issues = validation_results.get('issues', [])
    if issues:
        print(f"\nIssues Found ({len(issues)}):")
        for i, issue in enumerate(issues[:10], 1):
            print(f"  {i}. {issue}")


## 7. Key Insights & Recommendations

Tổng hợp insights và recommendations cho feature engineering.


In [ ]:
print("=" * 70)
print("KEY INSIGHTS & RECOMMENDATIONS")
print("=" * 70)

insights = []

if sales_df is not None:
    target_col = config['target_column']
    
    # Insight 1: Sparsity
    if 'PRODUCT_ID' in sales_df.columns or 'product_id' in sales_df.columns:
        product_col = 'PRODUCT_ID' if 'PRODUCT_ID' in sales_df.columns else 'product_id'
        time_col = config['time_column']
        
        # Check sparsity
        total_combinations = sales_df[product_col].nunique() * sales_df[time_col].nunique()
        actual_combinations = len(sales_df)
        sparsity = (1 - actual_combinations / total_combinations) * 100
        
        insights.append(f"📊 Data Sparsity: {sparsity:.1f}% missing combinations")
        insights.append("   → Recommendation: Use zero-filling in WS0 aggregation")
    
    # Insight 2: Seasonality
    if config['temporal_unit'] == 'hour':
        insights.append("📅 Hourly Patterns Detected")
        insights.append("   → Recommendation: Create hour-of-day, day-of-week features (WS2)")
    elif config['temporal_unit'] == 'week':
        insights.append("📅 Weekly Patterns Detected")
        insights.append("   → Recommendation: Create week-of-year, month features (WS2)")
    
    # Insight 3: Target distribution
    if target_col in sales_df.columns:
        zero_pct = (sales_df[target_col] == 0).sum() / len(sales_df) * 100
        if zero_pct > 20:
            insights.append(f"⚠️ High Zero Rate: {zero_pct:.1f}% zeros")
            insights.append("   → Recommendation: Consider zero-inflated models or separate zero/non-zero prediction")
        
        skewness = sales_df[target_col].skew()
        if abs(skewness) > 2:
            insights.append(f"📈 High Skewness: {skewness:.2f}")
            insights.append("   → Recommendation: Consider log transformation or robust scaling")

print("\n".join(insights))

print("\n" + "=" * 70)
print("NEXT STEPS")
print("=" * 70)
print("1. Run feature engineering pipeline (02_Feature_Engineering_Guide.ipynb)")
print("2. Train baseline models (baseline_model.ipynb)")
print("3. Train quantile models (03_Model_Training.ipynb)")
print("4. Generate predictions (04_Prediction_Forecasting.ipynb)")
